In [1]:
import os
import glob
import datetime as dt
import numpy as np
import pandas as pd
import pandas_ta as ta
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

from concurrent import futures
from dateutil.relativedelta import relativedelta
from pandas_datareader import data as pdr
from scipy.stats import gaussian_kde
# from talib import BBANDS, RSI, ICHIMOKU

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import plotly.graph_objects as go

""" datetime util """
now = dt.datetime.now()
lastday = now + relativedelta(months=0, days=-1)
firstday_of_this_month = dt.datetime(now.year, now.month, 1)
lastday_of_this_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=1, days=-1)
firstday_of_last_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=-1, days=0)
lastday_of_last_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=0, days=-1)

#import plotly.tools as tls
#import plotly.io as pio
#import plotly.graph_objs as go
#from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot

def last_working_day(given_date):
    while True:
        given_date -= dt.timedelta(days=1)
        if given_date.weekday() not in [5,6]:
            return given_date
        
""" set the download window """
start_date = "2017-01-01"
end_date = last_working_day(now).strftime('%Y-%m-%d')

""" set output directory """
data_dir = "./data/txn"
report_dir = "./report"
os.makedirs(data_dir, exist_ok=True)

In [2]:
# Load data
def loadTxnData(symbol, end_date):
    df = pd.DataFrame() # Empty DataFrame
    filename = f'./data/txn/{end_date}_{symbol}.csv'
    print(filename)
    if not os.path.exists(filename):
        df = pdr.get_data_yahoo(symbol)

        # Add new "name" column
        df.insert(0,'name',symbol)

        # Replace "datetime" with the appropriate column from your DataFrame
        df.rename(columns={"Date": "datetime", "Open": "open", "High": "high", "Low": "low", "Close": "close" , "Adj Close": "adj_close", "Volume": "volume" }, inplace=True) 
        df.index.names = ["datetime"]

        df.to_csv(filename)
    else:
        df = pd.read_csv(filename)

        # Replace "datetime" with the appropriate column from your DataFrame
        #df.rename(columns={"Date": "datetime", "Open": "open", "High": "high", "Low": "low", "Close": "close" , "Adj Close": "adj_close", "Volume": "volume" }, inplace=True)

        # VWAP requires the DataFrame index to be a DatetimeIndex.
        df.set_index(pd.DatetimeIndex(df["datetime"]), inplace=True)    
        del df['datetime']
    return df

In [3]:
# temp
# end_date="2023-01-27"

symbols = list(pd.read_csv(f"{report_dir}/most_attractives_{end_date}.csv")["stock"])


for symbol in symbols:
    df = loadTxnData(symbol, end_date)
    print(df.head(5))
    
# report_dir = "./report"
# filename = f"{report_dir}/most_attractives_{end_date}.csv"

# """ Load from CSV """
# df = pd.read_csv(filename)
# symbols = df["stock"].tolist()
# print(symbols)

FileNotFoundError: [Errno 2] No such file or directory: './report/most_attractives_2023-03-22.csv'

In [ ]:
for symbol in symbols:
    df = pd.read_csv(f"./data/txn/{end_date}_{symbol}.csv")
    print(df.head(5))